In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



## DATA


In [2]:
def load_data():
    ## preparing data
    # Prefix
    target_image_width = 160
    target_image_height = 130
    # parent dataset
    dataset_parent_folder = 'dataset'
    
    if not (os.path.exists(dataset_parent_folder) and os.path.isdir(dataset_parent_folder)):
        print(f"Error: Cant find the path {dataset_parent_folder}")
        raise FileNotFoundError(f"Dataset folder could not be found at {dataset_parent_folder}")
    
    print(f"Successfully accessed the parent dataset folder: {dataset_parent_folder}")
    
    # Subdirectories
    all_entries = os.listdir(dataset_parent_folder)
    # This line ensures all_dog_breed contains only names of actual directories, sorted
    all_dog_breed = sorted([entry for entry in all_entries if os.path.isdir(os.path.join(dataset_parent_folder, entry)) 
                                                            and not entry.startswith(".") ])
    
    if not all_dog_breed: # Check if the list is empty
        print(f"Warning: No breed subdirectories found in '{dataset_parent_folder}'.")
        raise FileNotFoundError("No breed subdirectories found to process.")
    else:
        print(f"Identified breed folders: {all_dog_breed}")
    
    num_of_breed = len(all_dog_breed) # Define num_of_breed
    print(f"Number of classes (breeds): {num_of_breed}")
    
    # one_hot_setup
    breed_to_idx = {breed_name:i for i, breed_name  in enumerate(all_dog_breed)}
    idx_to_breed = {i: breed_name for i, breed_name in enumerate(all_dog_breed)} # Useful for verification
    print(f"Breed to index mapping: {breed_to_idx}")
    
    # X Y (Initialize lists for all data)
    x_final = []
    y_final = []
    
    # Loop through the identified breed folder names
    for current_breed in all_dog_breed:
        current_breed_path = os.path.join(dataset_parent_folder, current_breed)
        print(f"\nProcessing breed: {current_breed}")
    
        # 1-hot-for the current breed
        one_hot_for_current_breed = np.zeros(num_of_breed, dtype=int) # Use dtype=int for one-hot
        current_breed_index = breed_to_idx[current_breed]
        one_hot_for_current_breed[current_breed_index] = 1
    
        files_in_current_breed = os.listdir(current_breed_path)
        loaded_count = 0 # To count images loaded for the current breed
    
        for file_name in files_in_current_breed:
            image_file_path = os.path.join(current_breed_path, file_name)
    
            # Check if the file is an image based on its extension
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                try:
                    # Open the image using Pillow
                    img = Image.open(image_file_path)
    
                    # Convert to RGB format for consistency
                    img = img.convert('RGB')
    
                    # Resize the image if it's not the target size
                    if img.size != (target_image_width, target_image_height):
                        img = img.resize((target_image_width, target_image_height))
    
                    # Append the image data (as NumPy array) and its one-hot label
                    x_final.append(np.array(img)) # Appending NumPy array of the image
                    y_final.append(one_hot_for_current_breed)
                    loaded_count +=1
    
                except Exception as e:
                    print(f"    Error loading or processing image '{image_file_path}': {e}")
        print(f"  Loaded {loaded_count} images for {current_breed}.")
    
    
    # Convert x y to NumPy arrays for easier use with ML frameworks
    if x_final and y_final: # Check if lists are not empty
        X_final_np_4D = np.array(x_final) # This creates the 4D array (num_images, height, width, channels)
        Y_final_np_2D = np.array(y_final) # This creates the 2D array (num_images, num_of_breed)
    
        print("\n--- Initial Array Shapes ---")
        print(f"Shape of X_final_np_4D (image data): {X_final_np_4D.shape}")
        print(f"Shape of Y_final_np_2D (labels): {Y_final_np_2D.shape}")
    
        # --- Reshaping X data for a dense layer (flattening each image) ---
        num_of_images = X_final_np_4D.shape[0] # Or simply len(x_final)
    
        # IMPORTANT CORRECTION: reshape() returns a new array (or a view).
        # You need to assign the result if you want to use the reshaped array.
        X_final_np_flattened = X_final_np_4D.reshape(num_of_images, -1) # -1 infers the flattened dimension size
    
        print("\n--- Reshaped Array Shape (for Dense Layer Input) ---")
        print(f"Shape of X_final_np_flattened (flattened image data): {X_final_np_flattened.shape}") # Should be (num_images, height*width*channels)
    
        # Now you have two versions of X:
        # X_final_np_4D: If you want to use CNNs (shape: num_images, height, width, channels)
        # X_final_np_flattened: If you want to use a simple dense layer (shape: num_images, features)
    
        # Print example of first few data points for verification
        if len(X_final_np_flattened) > 0: # Ensure there's data to print
            print("\nExample of first few loaded data points (using flattened X):")
            for i in range(min(5, num_of_images)):
                # np.argmax(Y_final_np_2D[i]) gives the integer index of the breed
                breed_index = np.argmax(Y_final_np_2D[i])
                print(f"  Flattened Image {i} feature count: {X_final_np_flattened[i].shape[0]}, Label (one-hot): {Y_final_np_2D[i]}, Breed Name: {idx_to_breed[breed_index]}")
    
    else:
        print("\n--- Image Loading Process Complete ---")
        print("No images were loaded, or no labels were created.")
    return X_final_np_flattened, Y_final_np_2D


In [21]:
X, Y = load_data()
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.4,random_state = 42)
X_test,X_validation,Y_test,Y_validation = train_test_split(X_test,Y_test,test_size = 0.5,random_state = 42)

m_train = X_train.shape[0]
print(X_train.shape,X_test.shape,X_validation.shape)

Successfully accessed the parent dataset folder: dataset
Identified breed folders: ['Beagle', 'Boxer', 'Dachshund', 'German_Shepherd', 'Golden_Retriever', 'Labrador_Retriever', 'Poodle', 'Rottweiler', 'Yorkshire_Terrier']
Number of classes (breeds): 9
Breed to index mapping: {'Beagle': 0, 'Boxer': 1, 'Dachshund': 2, 'German_Shepherd': 3, 'Golden_Retriever': 4, 'Labrador_Retriever': 5, 'Poodle': 6, 'Rottweiler': 7, 'Yorkshire_Terrier': 8}

Processing breed: Beagle
  Loaded 100 images for Beagle.

Processing breed: Boxer
  Loaded 100 images for Boxer.

Processing breed: Dachshund
  Loaded 96 images for Dachshund.

Processing breed: German_Shepherd
  Loaded 96 images for German_Shepherd.

Processing breed: Golden_Retriever
  Loaded 91 images for Golden_Retriever.

Processing breed: Labrador_Retriever
  Loaded 95 images for Labrador_Retriever.

Processing breed: Poodle
  Loaded 100 images for Poodle.

Processing breed: Rottweiler
  Loaded 89 images for Rottweiler.

Processing breed: Yorksh

## Foward


In [41]:
# a function
def relu(Z):
    return np.maximum(0,Z)
def linear(Z):
    return Z
def softmax(Z):
    Z_stabilized = Z - np.max(Z, axis= 1, keepdims=True)
    exp_Z = np.exp(Z_stabilized)

    sum_exp_Z = np.sum(exp_Z,axis=1, keepdims=True)

    return exp_Z / sum_exp_Z
    
def my_dense(A_in,W,b,g):
    """
    A_in (m,n) m:examples
               n: features
    W(n,j)     j: units
    b(1,j)
    """
    units = W.shape[1]
    Z = np.matmul(A_in,W)+b
    A_out = g(Z)
    return A_out
def my_sequential(X,W1,b1,g1,W2,b2,g2,W3,b3,g3):
    A1 = my_dense(X,W1,b1,g1)
    A2 = my_dense(A1,W2,b2,g2)
    A3 = my_dense(A2,W3,b3,g3)
    return A3

In [37]:
input_size = 62400
dense1_units = 30 # Dense1 30 units
dense2_units = 20 # Dense2 20 units
dense3_units = 9 # output 9 units

W1 = np.random.randn(input_size, dense1_units)*0.1
W2 = np.random.randn(dense1_units, dense2_units)*0.1
W3 = np.random.randn(dense2_units, dense3_units)*0.1

b1 = np.random.randn(1, dense1_units)*0.1
b2 = np.random.randn(1, dense2_units)*0.1
b3 = np.random.randn(1, dense3_units)*0.1


In [42]:
Prediction = my_sequential(X_train,W1,b1,relu,W2,b2,relu,W3,b3,softmax)
print(Prediction)

[[1.00000000e+000 6.57025249e-155 3.90734871e-283 ... 4.81193453e-295
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000 1.80015995e-185 0.00000000e+000 ... 8.58711029e-220
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000 1.65095001e-175 4.48883395e-302 ... 1.80797293e-229
  0.00000000e+000 0.00000000e+000]
 ...
 [1.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [1.53514399e-012 2.08426847e-018 3.04013421e-097 ... 3.44115556e-128
  5.68466919e-256 5.64371187e-320]
 [2.38446221e-001 2.45752679e-039 2.10298571e-166 ... 1.46287362e-082
  6.30052100e-229 6.32411663e-274]]
